# Analyse failed trades

Analyse failed trades over multiple trade executors.

 - Show failed and repaired and fix trades
  - Analyse slippage parameters and other causes of failure



In [1]:
monitored_executors = [
    #"https://enzyme-polygon-eth-usdc.tradingstrategy.ai",
    "https://enzyme-polygon-eth-usdc-sls.tradingstrategy.ai",
    "https://enzyme-polygon-matic-usdc.tradingstrategy.ai",
    # "https://enzyme-polygon-multipair.tradingstrategy.ai",
]



In [2]:
import requests
from tradeexecutor.state.state import State

total_downloaded = 0

def download_state(url) -> State:
    global total_downloaded
    resp = requests.get(f"{url}/state")

    if len(resp.content) == 0:
        raise RuntimeError(f"Could not download: {url}")

    try:
        state = State.read_json_blob(resp.text)
    except Exception as e:
        raise RuntimeError(f"Could not decode: {url}") from e
    total_downloaded += len(resp.content)
    return state

states = [download_state(url)for url in monitored_executors]


print(f"Downloaded {total_downloaded:,} bytes")

/Users/moo/Library/Caches/pypoetry/virtualenvs/trade-executor-8Oz1GdY1-py3.10/lib/python3.10/site-packages/dataclasses_json/core.py:184: RuntimeWarning: `NoneType` object value of non-optional type initial_cash detected when decoding TradeSummary.
  warnings.warn(


Downloaded 1,346,025 bytes


In [3]:
from tradeexecutor.state.trade import TradeExecution
from itertools import chain

# Get all trades across different trade executors
all_trades = chain(*[state.portfolio.get_all_trades() for state in states])

t: TradeExecution
failed_trades = [t for t in all_trades if t.is_failed() or t.is_repaired() or t.is_repair_trade()]

print(f"Total {len(failed_trades)} failed and repaired trades")

Total 17 failed and repaired trades


In [4]:
import pandas as pd
from tradeexecutor.analysis.slippage import display_slippage


pd.set_option('display.max_colwidth', None)

df = display_slippage(failed_trades)
display(df)

,Flags,Position,Trade,Started,Executed,Lag,Slippage tol (BPS),Tx,Notes,Failure reason
28,REP,#14,WETH->USDC,2023-07-05 21:00,2023-07-06 12:02,7.0,100.0,0x89633e…,"Repaired at 2023-07-06 12:02, by #29",execution reverted: ERC20: transfer amount exc...
29,FIX,#14,USDC->WETH,,2023-07-06 12:02,,,,Repairing trade #28,
30,REP,#14,WETH->USDC,2023-07-06 13:57,2023-07-07 08:39,,100.0,0x803327…,"Repaired at 2023-07-07 08:39, by #31",execution reverted: ERC20: transfer amount exc...
31,FIX,#14,USDC->WETH,,2023-07-07 08:39,,,,Repairing trade #30,
32,REP,#14,WETH->USDC,2023-07-07 08:42,2023-07-09 23:58,,100.0,0x2eca52…,"Repaired at 2023-07-09 23:58, by #33",execution reverted: ERC20: transfer amount exc...
33,FIX,#14,USDC->WETH,,2023-07-09 23:58,,,,Repairing trade #32,
34,REP,#14,WETH->USDC,2023-07-10 00:00,2023-07-10 00:03,,100.0,0xb5b249…,"Repaired at 2023-07-10 00:03, by #35",execution reverted: ERC20: transfer amount exc...
35,FIX,#14,USDC->WETH,,2023-07-10 00:03,,,,Repairing trade #34,
36,REP,#14,WETH->USDC,2023-07-10 00:06,2023-07-11 07:41,,100.0,0xbdb255…,"Repaired at 2023-07-11 07:41, by #37",execution reverted: ERC20: transfer amount exc...
37,FIX,#14,USDC->WETH,,2023-07-11 07:41,,,,Repairing trade #36,
